# Pitcher Data Demo

Inspired by: Thomas Nestico [(@TJStats)](https://x.com/TJStats)

The end result of this code will output a specific pitcher's outing from a specific game in Spring Training with specific values attributed to each type of pitch from the pitcher's pitch mix. This notebook will explain each attribute itself and how they are calculated. I wanted to try and replicate the incredible work that people like TJ put out for baseball fans so that I could better understand pitchers and the game we all love.

The end result will look as such:


In [774]:
import pandas as pd
pd.set_option("display.max_columns", None)  # Ensure all columns are displayed

df = pd.read_csv("pitch_type_counts.csv")

df

,Pitcher,Pitch Type,Count,Usage,Spin Rate,Avg Velo,iVB,HB,Whiffs,CS,CS+Whiffs,Zone%,Chase%,Whiff%,vRel,hRel,VAA,HAA,Extension,Max Exit Velo,Batter
0,Jon Gray,Four-Seam Fastball,20,50.0,1938.4,94.4,15.1,8.5,0,4,4,60.0,37.5,0.0,5.4,-1.6,-4.7,1.0,6.3,98.2,Lourdes Gurriel Jr.
1,Jon Gray,Slider,15,37.5,2473.7,86.7,1.9,-3.4,3,3,6,66.7,40.0,33.3,5.6,-1.6,-7.3,2.4,6.1,109.4,Alek Thomas
2,Jon Gray,Changeup,3,7.5,1502.7,87.3,8.1,14.2,1,0,1,66.7,0.0,50.0,5.5,-1.5,-6.7,0.6,6.3,NaN,NaN
3,Jon Gray,Curveball,2,5.0,2622.5,77.3,-8.3,-10.6,0,0,0,0.0,0.0,NaN,5.6,-1.5,-10.7,3.7,6.2,NaN,NaN


As we can see above, there are a lot of different attributes describing the pitches for Jon Gray.

We can see each pitch type that Jon has, as well as how many times he threw each pitch respectively (20 fastballs, 15 sliders, 3 changeups, 2 curveballs).

From that information we can calculate the next column, usage rate, by combining each pitch type count to give us a total amount of pitches thrown. With a total amount of pitches thrown, we can divide each pitch type count by the total to give us the usage rate.

I will go through this final output column by column and showcase the code used as well as an explanation for the code.

## Import Packages

In [775]:
# MLB Scraper Pitcher Data
import pandas as pd
import pybaseball as pyb
import numpy as np
from api_scraper import MLB_Scrape

y0 = 50  # Release y-position (feet)
yf = 17 / 12  # Home plate y-position (feet)

## Display Options to ensure that all of the output is displayed


In [776]:
# Set display options to print all columns without truncation
pd.set_option("display.max_columns", None)  # Ensure all columns are displayed
pd.set_option("display.max_rows", None)  # Display all rows, be cautious with large DataFrames
pd.set_option("display.width", None)  # Remove column width limit

## Retrieving game data with MLB Scraper model by Tnestico

Specific game IDs can be found in baseball savant URLs

For example: `https://baseballsavant.mlb.com/gamefeed?gamePk=778935`

The last six digits `778935` at the end of the URL is the gameID for a Rangers/Diamondbacks Spring Training Game

We are retrieving this data from `scraper.get_data(game_list_input=[778935])` and assigning the data to the variable `game_data`

The following line converts the retrieved game data (stored in `game_data`) into a Polars DataFrame and is then stored in the variable `data_df`

The last line converts the Polars DataFrame (`data_df`) to a Pandas DataFrame (`pandas_df`). This is necessary so that we can utilize Pandas' features later on.

In [777]:
# Initialize the scraper
scraper = MLB_Scrape()

# Retrieve game data for the specific game ID
game_data = scraper.get_data(game_list_input=[778935])

# Convert the game data to a Polars DataFrame
data_df = scraper.get_data_df(data_list=game_data)

# Convert the Polars DataFrame to a Pandas DataFrame
pandas_df = data_df.to_pandas()

This May Take a While. Progress Bar shows Completion of Data Retrieval.


Processing: 100%|██████████| 1/1 [00:00<00:00,  2.56iteration/s]

Converting Data to Dataframe.


We can now rename `pandas_df` to `df_pyb` for convenience's sake.

We will also print out the first few lines of the dataframe so that we can see all the data we have to work with.

In [778]:
df_pyb = pandas_df
df_pyb.head(5)

,game_id,game_date,batter_id,batter_name,batter_hand,batter_team,batter_team_id,pitcher_id,pitcher_name,pitcher_hand,pitcher_team,pitcher_team_id,ab_number,play_description,play_code,in_play,is_strike,is_swing,is_whiff,is_out,is_ball,is_review,pitch_type,pitch_description,strikes,balls,outs,strikes_after,balls_after,outs_after,start_speed,end_speed,sz_top,sz_bot,x,y,ax,ay,az,pfxx,pfxz,px,pz,vx0,vy0,vz0,x0,y0,z0,zone,type_confidence,plate_time,extension,spin_rate,spin_direction,vb,ivb,hb,launch_speed,launch_angle,launch_distance,launch_location,trajectory,hardness,hit_x,hit_y,index_play,play_id,start_time,end_time,is_pitch,type_type,type_ab,event,event_type,rbi,away_score,home_score
0,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Ball,B,False,False,None,None,None,False,False,FF,Four-Seam Fastball,0,0,0,0,1,0,94.1,86.6,3.117,1.573,152.25,166.36,-7.708112,28.376567,-15.040927,-4.023972,8.943871,-0.924854,2.682155,4.094442,-137.064647,-4.440241,-1.910395,50.002223,5.339861,11,0.90,0.398585,6.271919,1919,223,-15.1,15.6,6.5,NaN,NaN,NaN,None,None,None,NaN,NaN,3,ac357149-abaf-4e84-a61d-2d357c6f3cbb,2025-03-02T20:07:20.629Z,2025-03-02T20:07:24.856Z,True,pitch,None,None,None,NaN,NaN,NaN
1,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Foul,F,False,True,True,None,None,False,False,FF,Four-Seam Fastball,0,1,0,1,1,0,93.9,85.9,3.117,1.573,105.55,187.36,-7.708316,30.294774,-15.543560,-4.084714,8.812437,0.300314,1.904322,6.615272,-136.558642,-6.412778,-1.623706,50.001922,5.353759,9,0.89,0.401204,6.274797,1846,223,-16.3,14.8,6.0,NaN,NaN,NaN,None,None,None,NaN,NaN,4,ac71bd8c-9247-4005-99f7-d5cd76fa41b4,2025-03-02T20:07:33.617Z,2025-03-02T20:07:36.863Z,True,pitch,None,None,None,NaN,NaN,NaN
2,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Ball,B,False,False,None,None,None,False,False,CH,Changeup,1,1,0,1,2,0,87.2,80.0,3.117,1.573,87.60,217.06,-13.549664,26.297841,-23.706664,-8.354960,5.221508,0.771382,0.804411,8.219174,-126.646911,-6.721460,-1.433305,50.004169,5.393922,14,0.93,0.432790,6.248571,1589,231,-28.1,8.0,13.2,NaN,NaN,NaN,None,None,None,NaN,NaN,5,0a82af13-2c2f-4115-8c65-59d719c6df8a,2025-03-02T20:07:48.825Z,2025-03-02T20:07:53.717Z,True,pitch,None,None,None,NaN,NaN,NaN
3,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Called Strike,C,False,True,None,None,None,False,True,FF,Four-Seam Fastball,1,2,0,2,2,0,94.4,86.6,3.117,1.573,146.74,180.56,-11.289134,29.538970,-16.268626,-5.883129,8.289911,-0.780242,2.156177,4.872686,-137.420980,-5.538319,-1.809204,50.004103,5.297263,4,0.90,0.398087,6.171662,1931,227,-16.6,14.0,9.7,NaN,NaN,NaN,None,None,None,NaN,NaN,6,f87bfae7-95b2-458e-9326-bf90287f701c,2025-03-02T20:08:05.157Z,2025-03-02T20:08:32.174Z,True,pitch,None,None,None,NaN,NaN,NaN
4,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Swinging Strike,S,False,True,True,True,True,True,False,SL,Slider,2,2,0,3,2,0,88.1,81.2,3.117,1.573,100.08,198.92,4.219440,26.414158,-31.128767,2.535455,0.624380,0.443989,1.476112,4.219382,-128.186277,-4.207924,-1.552380,50.000087,5.568195,9,0.89,0.427243,6.200200,2443,137,-34.8,0.4,-5.5,NaN,NaN,NaN,None,None,None,NaN,NaN,7,35971e5e-9fc2-4d94-8e03-4d40000cb768,2025-03-02T20:08:34.915Z,2025-03-02T20:08:38.033Z,True,pitch,atBat,Strikeout,strikeout,0.0,0.0,0.0


If we print out the shape of `df_pyb` we can see how much data there is in this game.

In [779]:
df_pyb.shape

(304, 78)

If each row is thought of as one pitch, then there were 304 pitches thrown.

Since we are only looking to gather data for one specific pitcher, we can filter out the dataframe to only return rows (or pitches) where `pitcher_name` is equal to the pitcher's name.

For this notebook, we will be looking at the data from Jon Gray. Considering Jon Gray is a starting pitcher, we can assume that he would have one of the higher pitch counts for this game. This will result in a larger sample size for us to make calculations from.

In [813]:
df_pyb = df_pyb[(df_pyb["pitcher_name"] == "Jon Gray")]
print(df_pyb.shape)
df_pyb.head(5)

(40, 78)


,game_id,game_date,batter_id,batter_name,batter_hand,batter_team,batter_team_id,pitcher_id,pitcher_name,pitcher_hand,pitcher_team,pitcher_team_id,ab_number,play_description,play_code,in_play,is_strike,is_swing,is_whiff,is_out,is_ball,is_review,pitch_type,pitch_description,strikes,balls,outs,strikes_after,balls_after,outs_after,start_speed,end_speed,sz_top,sz_bot,x,y,ax,ay,az,pfxx,pfxz,px,pz,vx0,vy0,vz0,x0,y0,z0,zone,type_confidence,plate_time,extension,spin_rate,spin_direction,vb,ivb,hb,launch_speed,launch_angle,launch_distance,launch_location,trajectory,hardness,hit_x,hit_y,index_play,play_id,start_time,end_time,is_pitch,type_type,type_ab,event,event_type,rbi,away_score,home_score
0,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Ball,B,False,False,None,None,None,False,False,FF,Four-Seam Fastball,0,0,0,0,1,0,94.1,86.6,3.117,1.573,152.25,166.36,-7.708112,28.376567,-15.040927,-4.023972,8.943871,-0.924854,2.682155,4.094442,-137.064647,-4.440241,-1.910395,50.002223,5.339861,11,0.90,0.398585,6.271919,1919,223,-15.1,15.6,6.5,NaN,NaN,NaN,None,None,None,NaN,NaN,3,ac357149-abaf-4e84-a61d-2d357c6f3cbb,2025-03-02T20:07:20.629Z,2025-03-02T20:07:24.856Z,True,pitch,None,None,None,NaN,NaN,NaN
1,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Foul,F,False,True,True,None,None,False,False,FF,Four-Seam Fastball,0,1,0,1,1,0,93.9,85.9,3.117,1.573,105.55,187.36,-7.708316,30.294774,-15.543560,-4.084714,8.812437,0.300314,1.904322,6.615272,-136.558642,-6.412778,-1.623706,50.001922,5.353759,9,0.89,0.401204,6.274797,1846,223,-16.3,14.8,6.0,NaN,NaN,NaN,None,None,None,NaN,NaN,4,ac71bd8c-9247-4005-99f7-d5cd76fa41b4,2025-03-02T20:07:33.617Z,2025-03-02T20:07:36.863Z,True,pitch,None,None,None,NaN,NaN,NaN
2,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Ball,B,False,False,None,None,None,False,False,CH,Changeup,1,1,0,1,2,0,87.2,80.0,3.117,1.573,87.60,217.06,-13.549664,26.297841,-23.706664,-8.354960,5.221508,0.771382,0.804411,8.219174,-126.646911,-6.721460,-1.433305,50.004169,5.393922,14,0.93,0.432790,6.248571,1589,231,-28.1,8.0,13.2,NaN,NaN,NaN,None,None,None,NaN,NaN,5,0a82af13-2c2f-4115-8c65-59d719c6df8a,2025-03-02T20:07:48.825Z,2025-03-02T20:07:53.717Z,True,pitch,None,None,None,NaN,NaN,NaN
3,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Called Strike,C,False,True,None,None,None,False,True,FF,Four-Seam Fastball,1,2,0,2,2,0,94.4,86.6,3.117,1.573,146.74,180.56,-11.289134,29.538970,-16.268626,-5.883129,8.289911,-0.780242,2.156177,4.872686,-137.420980,-5.538319,-1.809204,50.004103,5.297263,4,0.90,0.398087,6.171662,1931,227,-16.6,14.0,9.7,NaN,NaN,NaN,None,None,None,NaN,NaN,6,f87bfae7-95b2-458e-9326-bf90287f701c,2025-03-02T20:08:05.157Z,2025-03-02T20:08:32.174Z,True,pitch,None,None,None,NaN,NaN,NaN
4,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Swinging Strike,S,False,True,True,True,True,True,False,SL,Slider,2,2,0,3,2,0,88.1,81.2,3.117,1.573,100.08,198.92,4.219440,26.414158,-31.128767,2.535455,0.624380,0.443989,1.476112,4.219382,-128.186277,-4.207924,-1.552380,50.000087,5.568195,9,0.89,0.427243,6.200200,2443,137,-34.8,0.4,-5.5,NaN,NaN,NaN,None,None,None,NaN,NaN,7,35971e5e-9fc2-4d94-8e03-4d40000cb768,2025-03-02T20:08:34.915Z,2025-03-02T20:08:38.033Z,True,pitch,atBat,Strikeout,strikeout,0.0,0.0,0.0


We can check the shape of the newly filtered DataFrame and can see that there are 40 rows in which `pitcher_name` is equal to Jon Gray.

We can also print out the first 5 lines of the DataFrame to see that we still have all the data we had originally, only that it now pertains specifically to Jon Gray.

There are a lot of columns that we do not need. If we want to find out Jon's pitch mix and how many times he threw each pitch, we can create a new DataFrame. This new DataFrame will contain the columns that we do want, and not ones that we don't want.

In [781]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
    ]
]
pitcher_pyb.head(5)

,game_id,game_date,pitcher_name,pitch_description
0,778935,2025-03-02,Jon Gray,Four-Seam Fastball
1,778935,2025-03-02,Jon Gray,Four-Seam Fastball
2,778935,2025-03-02,Jon Gray,Changeup
3,778935,2025-03-02,Jon Gray,Four-Seam Fastball
4,778935,2025-03-02,Jon Gray,Slider


Now we have a DataFrame that is still associated to Jon Gray, but is more concise and returns everything we need to calculate his total pitches thrown and find his usage rate for each pitch type.

One way to quickly sum the total pitches thrown is by creating a new column on the DataFrame named `PitchesThrown`.

In [782]:
pitcher_pyb["PitchesThrown"] = 1

/var/folders/dn/817fmqdd1331prc4rcwjcgqh0000gn/T/ipykernel_61686/2861990300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitcher_pyb["PitchesThrown"] = 1


Now with the `PitchesThrown` column created, we can create a DataFrame for pitch counts, total pitches, and a usage rate.

In [783]:
pitch_type_counts = pitcher_pyb.groupby("pitch_description", as_index=False)["PitchesThrown"].sum()
pitch_type_counts

,pitch_description,PitchesThrown
0,Changeup,3
1,Curveball,2
2,Four-Seam Fastball,20
3,Slider,15


We now have the amount of pitches thrown for each pitch type

We can sort the data by most pitches thrown for each pitch type

In [784]:
pitch_type_counts = pitch_type_counts.sort_values(by="PitchesThrown", ascending=False)
pitch_type_counts

,pitch_description,PitchesThrown
2,Four-Seam Fastball,20
3,Slider,15
0,Changeup,3
1,Curveball,2


Now we can calculate the total amout of pitches thrown

In [785]:
total_pitches = pitch_type_counts['PitchesThrown'].sum()
total_pitches

40

For the usage rate, we can create column `Usage` and divide `PitchesThrown` by `total_pitches`

We will also multiply `Usage` by 100 as well as round the answer to three decimal places

In [786]:
pitch_type_counts['Usage %'] = ((pitch_type_counts['PitchesThrown']/total_pitches)*100).round(3)
pitch_type_counts

,pitch_description,PitchesThrown,Usage %
2,Four-Seam Fastball,20,50.0
3,Slider,15,37.5
0,Changeup,3,7.5
1,Curveball,2,5.0


The next attribute to calculate is the average velocity for each pitch.

The first thing to do is update how we filtered the data originally. We need to add the column `start_speed`.

In [787]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
    ]
]

Now we can do what we previously did for `pitch_type_counts`.

Since `start_speed` is already a column in the raw data, we can simply use `.mean()` to find the average for each `pitch_description`. Additionally, we can round the average velocity to 1 decimal place.

In [788]:
pitch_type_velo = pitcher_pyb.groupby(['pitcher_name','pitch_description'],as_index=False)['start_speed'].mean().round(1)
pitch_type_velo

,pitcher_name,pitch_description,start_speed
0,Jon Gray,Changeup,87.3
1,Jon Gray,Curveball,77.3
2,Jon Gray,Four-Seam Fastball,94.4
3,Jon Gray,Slider,86.7


The next thing we must do is merge our `pitch_type_counts` DataFrame and our `pitch_type_velo` DataFrame so that we can use our velocity we calculated on `pitch_type_counts`.

In [789]:
pitch_type_counts = (pitch_type_counts.merge(pitch_type_velo, on="pitch_description", how="left"))

Now, when we print `pitch_type_counts` we have `start_speed` as a column for each `pitch_description`.

In [790]:
pitch_type_counts

,pitch_description,PitchesThrown,Usage %,pitcher_name,start_speed
0,Four-Seam Fastball,20,50.0,Jon Gray,94.4
1,Slider,15,37.5,Jon Gray,86.7
2,Changeup,3,7.5,Jon Gray,87.3
3,Curveball,2,5.0,Jon Gray,77.3


Next, we can calculate the average spin rate for each pitch type following the same pattern as before.

First step is to update `pitcher_pyb` and include `spin_rate` from the scraped data.

In [791]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
        "spin_rate"
    ]
]

After that, it is the same process as before when we calculated average velocity.

In [792]:
pitch_type_spin_rate = (pitcher_pyb.groupby("pitch_description", as_index=False)["spin_rate"].mean()).round(1)
pitch_type_spin_rate

,pitch_description,spin_rate
0,Changeup,1502.7
1,Curveball,2622.5
2,Four-Seam Fastball,1938.4
3,Slider,2473.7


Followed by a merging of `pitch_type_spin_rate` and `pitch_type_counts`.

We can also quickly re-order the columns in the output

In [793]:
pitch_type_counts = (pitch_type_counts.merge(pitch_type_spin_rate, on="pitch_description", how="left"))

pitch_type_counts = pitch_type_counts[
    [
        "pitcher_name",
        "pitch_description",
        "PitchesThrown",
        "Usage %",
        "start_speed",
        "spin_rate",
    ]
]

pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Usage %,start_speed,spin_rate
0,Jon Gray,Four-Seam Fastball,20,50.0,94.4,1938.4
1,Jon Gray,Slider,15,37.5,86.7,2473.7
2,Jon Gray,Changeup,3,7.5,87.3,1502.7
3,Jon Gray,Curveball,2,5.0,77.3,2622.5


## Induced Vertical Break (iVB)

The next value to calcuate is the Induced Vertical Break (iVB) for each pitch type.

Per [Fangraphs](https://blogs.fangraphs.com/a-visual-scouting-primer-pitching-part-two/), Induced Vertical Break aims to quantify a pitcher’s ability to fight gravity.

iVB does not require a calculation from our end as it is already listed in the raw data as `ivb`. We can simply update the `pitcher_pyb` to include this.

In [794]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
        "spin_rate",
        "ivb"
    ]
]

pitch_type_ivb = (pitcher_pyb.groupby("pitch_description", as_index=False)["ivb"].mean()).round(1)
pitch_type_ivb

,pitch_description,ivb
0,Changeup,8.1
1,Curveball,-8.3
2,Four-Seam Fastball,15.1
3,Slider,1.9


Horizontal break is the amount of lateral (side-to-side) movement a pitch experiences due to spin, measured in inches.

We can get the horizontal break the same way as Induced Vertical Break.

In [795]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
        "spin_rate",
        "ivb",
        "hb"
    ]
]

pitch_type_hb = (pitcher_pyb.groupby("pitch_description", as_index=False)["hb"].mean()).round(1)
pitch_type_hb

,pitch_description,hb
0,Changeup,14.2
1,Curveball,-10.6
2,Four-Seam Fastball,8.5
3,Slider,-3.4


Now, we can merge these two new DataFrames to `pitch_type_counts` and then see how our output is looking.

In [796]:
pitch_type_counts = (pitch_type_counts.merge(pitch_type_ivb, on="pitch_description", how="left"))
pitch_type_counts = (pitch_type_counts.merge(pitch_type_hb, on="pitch_description", how="left"))
pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Usage %,start_speed,spin_rate,ivb,hb
0,Jon Gray,Four-Seam Fastball,20,50.0,94.4,1938.4,15.1,8.5
1,Jon Gray,Slider,15,37.5,86.7,2473.7,1.9,-3.4
2,Jon Gray,Changeup,3,7.5,87.3,1502.7,8.1,14.2
3,Jon Gray,Curveball,2,5.0,77.3,2622.5,-8.3,-10.6


## vRel and hRel

The next piece of information to gather for our pitcher is their `vRel` and `hRel` values.

`vRel` is the vertical release position of the ball, measured in feet, from the catcher's perspective

Similarly, `hRel` is the horizontal release position of the ball, measured in feet, from the catcher's perspective

These values are listed as `z0` and `x0` in the raw data where `z0` is `vRel` and `x0` is `hRel`

We can update `pitcher_pyb` to include these values, create a new DataFrame for each value, calculate the mean of each value for each pitch type, check the output and merge the DataFrames to `pitch_type_counts`. Additionally, we can rename `z0` and `x0` to `vRel` and `hRel` respectively.

In [797]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
        "spin_rate",
        "ivb",
        "hb",
        "z0",
        "x0"
    ]
]

In [798]:
pitch_type_vrel = (pitcher_pyb.groupby("pitch_description", as_index=False)["z0"].mean()).round(1)
pitch_type_vrel.rename(columns={"z0": "vRel"}, inplace=True)
pitch_type_vrel

,pitch_description,vRel
0,Changeup,5.5
1,Curveball,5.6
2,Four-Seam Fastball,5.4
3,Slider,5.6


In [799]:
pitch_type_hrel = (pitcher_pyb.groupby("pitch_description", as_index=False)["x0"].mean()).round(1)
pitch_type_hrel.rename(columns={"x0": "hRel"}, inplace=True)
pitch_type_hrel

,pitch_description,hRel
0,Changeup,-1.5
1,Curveball,-1.5
2,Four-Seam Fastball,-1.6
3,Slider,-1.6


In [800]:
pitch_type_counts = (pitch_type_counts.merge(pitch_type_vrel, on="pitch_description", how="left"))
pitch_type_counts = (pitch_type_counts.merge(pitch_type_hrel, on="pitch_description", how="left"))
pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Usage %,start_speed,spin_rate,ivb,hb,vRel,hRel
0,Jon Gray,Four-Seam Fastball,20,50.0,94.4,1938.4,15.1,8.5,5.4,-1.6
1,Jon Gray,Slider,15,37.5,86.7,2473.7,1.9,-3.4,5.6,-1.6
2,Jon Gray,Changeup,3,7.5,87.3,1502.7,8.1,14.2,5.5,-1.5
3,Jon Gray,Curveball,2,5.0,77.3,2622.5,-8.3,-10.6,5.6,-1.5


## Whiffs

One thing that is nice to know about a pitcher's performance is how many whiffs they generated.

A whiff refers to a swing and miss (when a batter swings at a pitch but fails to make contact.)

Pitchers with a lot of whiffs (or a high Whiff%, a metric we will calculate later) tend to be effective strikeout pitchers, as more swings and misses (whiffs) lead to more strikeouts.

Fortunately, the raw data has a column called `is_whiff` has the value `True` when a swing and miss occurs, and `NaN` when a whiff did not occur.

We can add `is_whiff` to `pitcher_pyb` and create a new DataFrame `whiff_pitches` that returns the rows where a whiff has occured.

In [801]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
        "spin_rate",
        "ivb",
        "hb",
        "z0",
        "x0",
        "is_whiff"
    ]
]

whiff_pitches = pitcher_pyb[pitcher_pyb["is_whiff"] == True]
whiff_pitches

,game_id,game_date,pitcher_name,pitch_description,start_speed,spin_rate,ivb,hb,z0,x0,is_whiff
4,778935,2025-03-02,Jon Gray,Slider,88.1,2443,0.4,-5.5,5.568195,-1.552380,True
16,778935,2025-03-02,Jon Gray,Slider,87.2,2507,3.0,-3.0,5.451645,-1.622137,True
17,778935,2025-03-02,Jon Gray,Slider,87.7,2441,-1.7,-4.4,5.433623,-1.516443,True
67,778935,2025-03-02,Jon Gray,Changeup,87.5,1521,6.5,15.6,5.585674,-1.589213,True


From here we can then create a `pitch_type_whiff` DataFrame that calculates the total amount of whiffs for each pitch type and merge this DataFrame to `pitch_type_counts`

In [802]:
pitch_type_whiff = (
    whiff_pitches.groupby("pitch_description").size().reset_index(name="Whiffs")
)
pitch_type_whiff

,pitch_description,Whiffs
0,Changeup,1
1,Slider,3


We know that Gray threw 4 pitches this day, but only 2 of his 4 pitches generated whiffs. We can assume that the other half of his arsenal did not generate any whiffs for this outing.

During the merge to `pitch_type_counts` we can replace `NaN` with 0, and ensure the pitch types that did generate whiffs have the whiffs represented as an integer as a pitcher can only have a whole number of whiffs.

In [803]:
pitch_type_counts = (
    pitch_type_counts.merge(pitch_type_whiff, on="pitch_description", how="left")
    .fillna({"Whiffs": 0})
    .astype({"Whiffs": int})
)
pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Usage %,start_speed,spin_rate,ivb,hb,vRel,hRel,Whiffs
0,Jon Gray,Four-Seam Fastball,20,50.0,94.4,1938.4,15.1,8.5,5.4,-1.6,0
1,Jon Gray,Slider,15,37.5,86.7,2473.7,1.9,-3.4,5.6,-1.6,3
2,Jon Gray,Changeup,3,7.5,87.3,1502.7,8.1,14.2,5.5,-1.5,1
3,Jon Gray,Curveball,2,5.0,77.3,2622.5,-8.3,-10.6,5.6,-1.5,0


## Whiff%

Knowing the amount of whiffs generated is important, but another metric that is useful is a pitchers Whiff%.

Whiff% measures how often a batter swings and misses at a pitch, relative to their total swings. It is a key stat for evaluating a pitcher's ability to generate swings and misses.

Since we already know how many whiffs there are for each pitch type, we simply need to calculate the total amount of swings for each pitch type.

Once we have the total amount of swings, calculating Whiff% is quite simple.

Similar to `is_whiff`, `is_swing` exists within our data and also has `True` or `NaN` to indicate when a batter swung.

We can add `is_swing` to `pitcher_pyb` and use the same logic behind calculating `Whiffs` to calculate `Whiff%`.

In [804]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "pitch_description",
        "start_speed",
        "spin_rate",
        "ivb",
        "hb",
        "z0",
        "x0",
        "is_whiff",
        "is_swing"
    ]
]

Here we are creating a DataFrame `swings` that will return every row in which `is_swing` is True

In [805]:
swings = pitcher_pyb[pitcher_pyb["is_swing"] == True]

Now we are grouping each pitch type with the total amount of swings each pitch type generated

In [806]:
pitch_type_swing = (
    swings.groupby("pitch_description").size().reset_index(name="Swings")
)
pitch_type_swing

,pitch_description,Swings
0,Changeup,2
1,Four-Seam Fastball,11
2,Slider,9


Since there appears to be one pitch type that did not generate >0 swings, we can replace the NaN value it would return with a 0 and ensure that swings are represented as an integer.

In [807]:
pitch_type_counts = (
    pitch_type_counts.merge(pitch_type_swing, on="pitch_description", how="left")
    .fillna({"Swings": 0})
    .astype({"Swings": int})
)

pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Usage %,start_speed,spin_rate,ivb,hb,vRel,hRel,Whiffs,Swings
0,Jon Gray,Four-Seam Fastball,20,50.0,94.4,1938.4,15.1,8.5,5.4,-1.6,0,11
1,Jon Gray,Slider,15,37.5,86.7,2473.7,1.9,-3.4,5.6,-1.6,3,9
2,Jon Gray,Changeup,3,7.5,87.3,1502.7,8.1,14.2,5.5,-1.5,1,2
3,Jon Gray,Curveball,2,5.0,77.3,2622.5,-8.3,-10.6,5.6,-1.5,0,0


Now that we have both `Whiffs` and `Swings`, we can calulate `Whiff%` as the amount of `Whiffs` divided by the amount of `Swings`.

If a pitch type did not have any swings, then the result will be `NaN` since it contain division by zero.

In [808]:
pitch_type_counts["Whiff%"] = (
    (pitch_type_counts["Whiffs"] / pitch_type_counts["Swings"]) * 100
).round(1)

pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Usage %,start_speed,spin_rate,ivb,hb,vRel,hRel,Whiffs,Swings,Whiff%
0,Jon Gray,Four-Seam Fastball,20,50.0,94.4,1938.4,15.1,8.5,5.4,-1.6,0,11,0.0
1,Jon Gray,Slider,15,37.5,86.7,2473.7,1.9,-3.4,5.6,-1.6,3,9,33.3
2,Jon Gray,Changeup,3,7.5,87.3,1502.7,8.1,14.2,5.5,-1.5,1,2,50.0
3,Jon Gray,Curveball,2,5.0,77.3,2622.5,-8.3,-10.6,5.6,-1.5,0,0,NaN


In [809]:
'''pitcher_pyb = pitcher_pyb.sort_values(by=["pitch_description"])
is_ball = [11, 12, 13, 14]
strike = [1, 2, 3, 4, 5, 6, 7, 8, 9]
pitcher_pyb["InZone"] = np.where(pitcher_pyb["zone"].isin(strike), 1, 0)
pitcher_pyb["OutZone"] = np.where(pitcher_pyb["zone"].isin(is_ball), 1, 0)
pitcher_pyb'''

'''pitcher_pyb["vy_f"] = -np.sqrt(
    pitcher_pyb["vy0"] ** 2 - (2 * pitcher_pyb["ay"] * (y0 - yf))
)

# Compute time (t)
pitcher_pyb["t"] = (pitcher_pyb["vy_f"] - pitcher_pyb["vy0"]) / pitcher_pyb["ay"]

# Compute final z-velocity (vz_f)
pitcher_pyb["vz_f"] = pitcher_pyb["vz0"] + (pitcher_pyb["az"] * pitcher_pyb["t"])

# Compute final x-velocity (vx_f)
pitcher_pyb["vx_f"] = pitcher_pyb["vx0"] + (pitcher_pyb["ax"] * pitcher_pyb["t"])

# Compute VAA
pitcher_pyb["VAA"] = -np.arctan(pitcher_pyb["vz_f"] / pitcher_pyb["vy_f"]) * (
    180 / np.pi
)

# Compute Horizontal Approach Angle (HAA)
pitcher_pyb["HAA"] = -np.arctan(pitcher_pyb["vx_f"] / pitcher_pyb["vy_f"]) * (
    180 / np.pi
)

# Get average vRel per pitch type
pitch_type_vrel = (
    df_pyb.groupby("pitch_description", as_index=False)["z0"].mean()
).round(1)
pitch_type_vrel.rename(columns={"z0": "vRel"}, inplace=True)

# Get average hRel per pitch type
pitch_type_hrel = (
    df_pyb.groupby("pitch_description", as_index=False)["x0"].mean()
).round(1)
pitch_type_hrel.rename(columns={"x0": "hRel"}, inplace=True)
pitcher_hand_unique = df_pyb[["pitch_description", "pitcher_hand"]].drop_duplicates(
    subset=["pitch_description"]
)
pitch_type_hrel = pitch_type_hrel.merge(
    pitcher_hand_unique, on="pitch_description", how="left"
)
pitch_type_hrel["hRel"] = np.where(
    pitch_type_hrel["pitcher_hand"] == "L",
    -pitch_type_hrel["hRel"],
    pitch_type_hrel["hRel"],
)

whiff_pitches = pitcher_pyb[pitcher_pyb["is_whiff"] == True]
pitch_type_whiff = (
    whiff_pitches.groupby("pitch_description").size().reset_index(name="whiff_count")
)
pitch_type_whiff.rename(columns={"whiff_count": "Whiffs"}, inplace=True)
pitch_type_whiff["Whiffs"] = pitch_type_whiff["Whiffs"].astype(int)

strike_pitches = pitcher_pyb[pitcher_pyb["play_code"] == "C"]
pitch_type_cs = (
    strike_pitches.groupby("pitch_description").size().reset_index(name="CS")
)

pitches_in_zone = pitcher_pyb.groupby("pitch_description")["InZone"].sum().reset_index()
pitches_in_zone.rename(columns={"InZone": "Pitches_In_Zone"}, inplace=True)

pitches_out_of_zone = (
    pitcher_pyb.groupby("pitch_description")["OutZone"].sum().reset_index()
)
pitches_out_of_zone.rename(columns={"OutZone": "Pitches_Out_Of_Zone"}, inplace=True)
# print(pitches_out_of_zone)


swings_out_of_zone = pitcher_pyb[
    pitcher_pyb["zone"].isin([11, 12, 13, 14]) & pitcher_pyb["is_swing"] == True
]
swings_out_of_zone = (
    swings_out_of_zone.groupby("pitch_description")["is_swing"]
    .sum()
    .astype(int)  # Convert to integer
    .reset_index()
)
# print(swings_out_of_zone)


pitch_type_spin = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["spin_rate"].mean()
).round(1)
pitch_type_spin.rename(columns={"spin_rate": "Spin Rate"}, inplace=True)

pitch_type_swing = (
    pitcher_pyb.groupby("pitch_description")["is_swing"].sum().astype(int).reset_index()
)
pitch_type_swing.rename(columns={"is_swing": "Swings"}, inplace=True)


pitch_type_ivb = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["ivb"].mean()
).round(1)
pitch_type_ivb.rename(columns={"ivb": "iVB"}, inplace=True)

pitch_type_hb = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["hb"].mean()
).round(1)
pitch_type_hb.rename(columns={"hb": "HB"}, inplace=True)

pitch_avg_velo = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["start_speed"].mean()
).round(1)
pitch_avg_velo.rename(columns={"start_speed": "Avg Velo"}, inplace=True)

pitch_avg_exten = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["extension"].mean()
).round(1)
pitch_avg_exten.rename(columns={"extension": "Extension"}, inplace=True)

# Compute the mean VAA for each pitch type
vaa_means = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["VAA"].mean()
).round(1)

# Compute the mean HAA for each pitch type
haa_means = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["HAA"].mean()
).round(1)

# Compute the highest exit velocity for each pitch type
df_hits = pitcher_pyb.dropna(subset=["launch_speed"])
# Group by pitch type and find the index of the max exit velocity, handling NaN values
idx = df_hits.groupby("pitch_description")["launch_speed"].idxmax().dropna()
# Retrieve the rows with max exit velocity
max_exit_velo = df_hits.loc[
    idx, ["pitch_description", "batter_name", "launch_speed"]
].copy()
max_exit_velo.rename(columns={"launch_speed": "Max Exit Velo"}, inplace=True)

pitch_type_counts = pitcher_pyb.groupby(
    ["pitcher_name", "pitch_description"], as_index=False
)["PitchesThrown"].sum()'''

'pitcher_pyb["vy_f"] = -np.sqrt(\n    pitcher_pyb["vy0"] ** 2 - (2 * pitcher_pyb["ay"] * (y0 - yf))\n)\n\n# Compute time (t)\npitcher_pyb["t"] = (pitcher_pyb["vy_f"] - pitcher_pyb["vy0"]) / pitcher_pyb["ay"]\n\n# Compute final z-velocity (vz_f)\npitcher_pyb["vz_f"] = pitcher_pyb["vz0"] + (pitcher_pyb["az"] * pitcher_pyb["t"])\n\n# Compute final x-velocity (vx_f)\npitcher_pyb["vx_f"] = pitcher_pyb["vx0"] + (pitcher_pyb["ax"] * pitcher_pyb["t"])\n\n# Compute VAA\npitcher_pyb["VAA"] = -np.arctan(pitcher_pyb["vz_f"] / pitcher_pyb["vy_f"]) * (\n    180 / np.pi\n)\n\n# Compute Horizontal Approach Angle (HAA)\npitcher_pyb["HAA"] = -np.arctan(pitcher_pyb["vx_f"] / pitcher_pyb["vy_f"]) * (\n    180 / np.pi\n)\n\n# Get average vRel per pitch type\npitch_type_vrel = (\n    df_pyb.groupby("pitch_description", as_index=False)["z0"].mean()\n).round(1)\npitch_type_vrel.rename(columns={"z0": "vRel"}, inplace=True)\n\n# Get average hRel per pitch type\npitch_type_hrel = (\n    df_pyb.groupby("pitch_des

In [810]:
'''pitch_type_counts'''

'pitch_type_counts'

In [811]:
'''pitch_type_counts["Total Pitches"] = pitch_type_counts["PitchesThrown"].sum()
pitch_type_counts'''

'pitch_type_counts["Total Pitches"] = pitch_type_counts["PitchesThrown"].sum()\npitch_type_counts'

In [812]:
'''pitch_type_counts["Usage"] = (
    (pitch_type_counts["PitchesThrown"] / pitch_type_counts["Total Pitches"]) * 100
).round(2)

pitch_type_counts = (
    pitch_type_counts.merge(pitch_type_spin, on="pitch_description", how="left")
    .merge(pitch_avg_velo, on="pitch_description", how="left")
    .merge(pitch_type_ivb, on="pitch_description", how="left")
    .merge(pitch_type_hb, on="pitch_description", how="left")
    .merge(pitch_type_whiff, on="pitch_description", how="left")
    .merge(pitch_type_cs, on="pitch_description", how="left")
    .merge(pitches_in_zone, on="pitch_description", how="left")
    .merge(pitches_out_of_zone, on="pitch_description", how="left")
    .merge(swings_out_of_zone, on="pitch_description", how="left")
    .merge(pitch_type_swing, on="pitch_description", how="left")
    .merge(pitch_type_vrel, on="pitch_description", how="left")
    .merge(pitch_type_hrel, on="pitch_description", how="left")
    .merge(vaa_means, on="pitch_description", how="left")
    .merge(haa_means, on="pitch_description", how="left")
    .merge(pitch_avg_exten, on="pitch_description", how="left")
    .merge(max_exit_velo, on="pitch_description", how="left")
)

pitch_type_counts = pitch_type_counts.sort_values(by="PitchesThrown", ascending=False)

pitch_type_counts["Whiffs"] = pitch_type_counts["Whiffs"].fillna(0).astype(int)
pitch_type_counts["CS"] = pitch_type_counts["CS"].fillna(0).astype(int)
pitch_type_counts["CS+Whiffs"] = pitch_type_counts["CS"] + pitch_type_counts["Whiffs"]
pitch_type_counts["Zone%"] = (
    (pitch_type_counts["Pitches_In_Zone"] / pitch_type_counts["PitchesThrown"]) * 100
).round(1)
pitch_type_counts["is_swing"] = pitch_type_counts["is_swing"].fillna(0).astype(int)
pitch_type_counts["Chase%"] = (
    (pitch_type_counts["is_swing"] / pitch_type_counts["Pitches_Out_Of_Zone"]) * 100
).round(1)
pitch_type_counts["Whiff%"] = (
    (pitch_type_counts["Whiffs"] / pitch_type_counts["Swings"]) * 100
).round(1)


pitch_type_counts = pitch_type_counts[
    [
        "pitcher_name",
        "pitch_description",
        "PitchesThrown",
        "Usage",
        "Spin Rate",
        "Avg Velo",
        "iVB",
        "HB",
        "Whiffs",
        "CS",
        "CS+Whiffs",
        "Zone%",
        "Chase%",
        "Whiff%",
        "vRel",
        "hRel",
        "VAA",
        "HAA",
        "Extension",
        "game_date",
    ]
]


pitch_type_counts.rename(
    columns={
        "game_date": "Date",
        "PitchesThrown": "Count",
        "pitch_description": "Pitch Type",
        "batter_name": "Batter",
        "pitcher_name": "Pitcher",
    },
    inplace=True,
)


print(pitch_type_counts)
'''

'pitch_type_counts["Usage"] = (\n    (pitch_type_counts["PitchesThrown"] / pitch_type_counts["Total Pitches"]) * 100\n).round(2)\n\npitch_type_counts = (\n    pitch_type_counts.merge(pitch_type_spin, on="pitch_description", how="left")\n    .merge(pitch_avg_velo, on="pitch_description", how="left")\n    .merge(pitch_type_ivb, on="pitch_description", how="left")\n    .merge(pitch_type_hb, on="pitch_description", how="left")\n    .merge(pitch_type_whiff, on="pitch_description", how="left")\n    .merge(pitch_type_cs, on="pitch_description", how="left")\n    .merge(pitches_in_zone, on="pitch_description", how="left")\n    .merge(pitches_out_of_zone, on="pitch_description", how="left")\n    .merge(swings_out_of_zone, on="pitch_description", how="left")\n    .merge(pitch_type_swing, on="pitch_description", how="left")\n    .merge(pitch_type_vrel, on="pitch_description", how="left")\n    .merge(pitch_type_hrel, on="pitch_description", how="left")\n    .merge(vaa_means, on="pitch_description",